In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("raw_data/benchmark_attention_results.csv")
df = df.rename(columns={"context_length": "Context", "forward_mean": "forward", "backward_mean": "backward", "memory_mean": "memory", "d_model": "Dimension"})\
    [["compile", "Dimension", "Context", "forward_only", "forward", "backward", "memory"]]
df["forward"] = df["forward"] * 1000
df["backward"] = df["backward"] * 1000
df["memory"] = df["memory"] / (1024 ** 2)

In [ ]:
def prepare_report(df1, col):
    eager = df1[~df1.compile].pivot_table(
        index="Dimension", columns="Context", values=col, observed=False,
    )
    compiled = df1[df1.compile].pivot_table(
        index="Dimension", columns="Context", values=col, observed=False,
    )

    # Align and combine into the formatted string
    report = eager.combine(
        compiled,
        lambda a, b: a.combine(
        b,
        lambda x, y: f"{x:.1f} / {y:.1f}" if pd.notna(x) and pd.notna(y)
                    else f"{x:.1f} / OOM" if pd.notna(x)
                    else f"OOM / {y:.1f}" if pd.notna(y)
                    else "OOM / OOM"
       ),
    )

    return report

In [ ]:
forward_infer = prepare_report(df[df.forward_only], "forward")
backward = prepare_report(df[~df.forward_only], "backward")
memory = prepare_report(df[~df.forward_only], "memory")

In [ ]:
def print_report(df2, title):
    tex = df2.to_latex(index=True, caption=f"{title} (eager / compiled)", escape=False)
    tex = tex.replace(r"\begin{table}", r"\begin{table}[H]")
    print(tex)

In [ ]:
print_report(forward_infer, "Forward time in ms")
print_report(backward, "Backward time in ms")
print_report(memory, "Memory size in GB")